In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [2]:
post = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\FDATA_Post_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-53-59_radiomics_features_ALL.csv", index_col=0) 
post.set_index('pt_no', inplace=True)
current_post = post.columns.tolist()
new_post = {col: 'post_' + col for col in current_post}
post = post.rename(columns=new_post)
post

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,post_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,post_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,post_wavelet-LLL_glszm_ZoneEntropy,post_wavelet-LLL_glszm_ZonePercentage,post_wavelet-LLL_glszm_ZoneVariance,post_wavelet-LLL_ngtdm_Busyness,post_wavelet-LLL_ngtdm_Coarseness,post_wavelet-LLL_ngtdm_Complexity,post_wavelet-LLL_ngtdm_Contrast,post_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,2.212185,0.035648,3.621928,0.003134,1.429334e+06,15.156426,0.003068,0.932926,0.002431,0.037266
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,2.399476,0.012666,5.709832,0.003814,5.948491e+05,14.851495,0.002102,1.266959,0.005343,0.028783
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,3.027236,0.200319,4.477880,0.000856,1.198165e+07,74.174093,0.000500,1.435507,0.010100,0.008668
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,6.880579,0.013353,3.509561,0.001170,1.839469e+07,71.012121,0.000532,1.616311,0.008991,0.006797
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,5.894270,0.043460,3.628726,0.002132,2.952561e+06,148.316482,0.000469,3.045954,0.013363,0.004109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.490065,0.490065,2.404010,0.001197,6.828967e+06,12.033645,0.021085,0.010323,0.000106,0.019846
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,4.318118,0.093450,4.398636,0.001724,2.190314e+07,92.347019,0.000635,0.615894,0.001000,0.004040
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,6.316938,0.040322,4.143625,0.000649,6.753192e+07,40.683255,0.000882,0.522524,0.002792,0.009614


In [3]:
delta = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\FDATA_DELTA.csv", index_col=0) 
current_delta = delta.columns.tolist()
new_delta = {col: 'delta_' + col for col in current_delta}
delta = delta.rename(columns=new_delta)
delta

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,-2.939604,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,-0.346479,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,-0.268550,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,-3.072244,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,-1.006784,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.297115,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.404555,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869


In [4]:
ypCR_data = pd.read_excel(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx", index_col=0) 
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [5]:
data = pd.concat([post, delta], axis=1)
data

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,-2.939604,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,-0.346479,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,-0.268550,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,-3.072244,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,-1.006784,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.297115,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,-0.404555,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869


In [6]:
data['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
data

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,10.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,80.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,10.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,100.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,10.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,50.0


In [7]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# post['encoded_PDL1'] = pd.cut(post['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# post
conditions = [
    (data['post_PDL1_CPS'] >= 0) & (data['post_PDL1_CPS'] < 1),
    (data['post_PDL1_CPS'] >= 1) & (data['post_PDL1_CPS'] <= 10),
    (data['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
data['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
data

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,10.0,1.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,80.0,2.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,10.0,1.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,100.0,2.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,10.0,1.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,50.0,2.0


In [8]:
data.drop(columns=['post_PDL1_CPS'], inplace=True)
data

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,1.0
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,2.0
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,1.0
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,2.0
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,1.0
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,2.0


In [9]:
feature = data.drop(columns='encoded_PDL1')
target = data['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['post_exponential_firstorder_90Percentile',
 'post_exponential_firstorder_Energy',
 'post_exponential_firstorder_Entropy',
 'post_exponential_firstorder_InterquartileRange',
 'post_exponential_firstorder_Kurtosis',
 'post_exponential_firstorder_Maximum',
 'post_exponential_firstorder_Mean',
 'post_exponential_firstorder_MeanAbsoluteDeviation',
 'post_exponential_firstorder_Median',
 'post_exponential_firstorder_Minimum',
 'post_exponential_firstorder_Range',
 'post_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'post_exponential_firstorder_RootMeanSquared',
 'post_exponential_firstorder_Skewness',
 'post_exponential_firstorder_TotalEnergy',
 'post_exponential_firstorder_Uniformity',
 'post_exponential_firstorder_Variance',
 'post_exponential_glcm_Autocorrelation',
 'post_exponential_glcm_ClusterProminence',
 'post_exponential_glcm_ClusterShade',
 'post_exponential_glcm_ClusterTendency',
 'post_exponential_glcm_Contrast',
 'post_exponential_glcm_Correlation',
 'post_exponential_

In [10]:
feature

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,2.219483,29461.901076,2.260772e-01,0.064347,34.266328,2.570182,2.146833,0.051414,2.154137,1.001278,...,-2.939604,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340
17815426,1.686390,41570.297716,4.008505e-02,0.127298,4.996606,2.588496,1.488268,0.196380,1.583833,0.703832,...,-0.346479,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841
20109646,1.938055,137795.397861,2.976619e-02,0.312276,4.426818,2.047468,1.642503,0.234362,1.699540,0.726375,...,-0.268550,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723
32221501,1.754008,35219.185753,-3.203427e-16,0.090460,16.511103,1.994726,1.665865,0.056021,1.667869,0.730864,...,-3.072244,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.765510,27263.494816,-3.203427e-16,0.045081,15.363585,1.839605,1.722063,0.027728,1.723782,1.237867,...,-1.006784,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004
56124435,1.618351,112894.041187,-3.203427e-16,0.045193,40.283538,1.729826,1.568765,0.031478,1.571385,0.843665,...,0.297115,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373
56131318,1.547055,126669.162388,5.433248e-04,0.040824,6.975010,2.006795,1.436304,0.140364,1.514411,0.717912,...,-0.404555,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869


In [11]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [13]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

In [14]:
X_train

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,1.627911,24309.456841,-3.203427e-16,0.108932,6.017983,1.938772,1.520828,0.068795,1.523748,0.959160,...,3.712660,0.045608,1.525231,0.008755,-3.789789e+06,44.931031,-0.007340,0.149927,0.000608,-0.043247
37567637,1.483303,28805.828092,3.017787e-03,0.022169,90.191127,2.147837,1.460514,0.016428,1.463005,1.042888,...,-2.767768,0.218069,-0.639354,-0.008054,1.952179e+07,-7.709710,0.020640,-1.845684,-0.011488,0.005915
49087653,1.560215,47505.631970,-3.203427e-16,0.042323,52.445595,1.638994,1.511106,0.034672,1.519543,0.721676,...,-2.105801,-0.015490,-0.851733,-0.002741,3.583023e+07,-10.389029,0.004871,-2.040553,-0.006333,0.051593
46695156,1.844094,27042.952113,4.040453e-03,0.156254,6.138386,2.062806,1.648638,0.168702,1.727501,0.751675,...,-13.651342,-0.014117,-1.188595,-0.010801,2.913572e+06,19.467073,-0.007118,-6.084096,0.011340,-1.422193
54502130,1.779143,50327.034940,1.160113e-01,0.057897,26.569787,2.702452,1.667244,0.062655,1.659407,0.733516,...,4.448422,-0.009817,-0.764656,-0.002872,5.336440e+06,36.094545,-0.013138,2.579153,0.002983,-0.164958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,2.583926,53839.454637,9.505661e-01,0.664898,3.372705,3.598932,1.919339,0.380019,1.719894,1.393330,...,-3.931039,-0.116855,-0.175324,-0.002942,8.552021e+05,62.221669,-0.000776,-1.385447,0.009070,-0.024865
37509338,1.551646,12486.258508,-3.203427e-16,0.066923,14.469930,1.658007,1.472264,0.069450,1.502731,0.689920,...,0.639735,0.010700,3.000000,0.000345,3.086606e+06,6.305412,-999999.990676,0.292578,0.000762,0.048651
50740947,1.750991,75489.240704,6.163147e-04,0.041078,37.285483,2.186265,1.695314,0.048156,1.711922,0.778814,...,-2.627177,-0.051192,1.712897,-0.000592,1.976091e+07,7.873596,-0.012838,0.428536,0.000517,-0.141541


In [15]:
X_test

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
42748515,1.825962,35553.563074,1.503103e-02,0.115020,14.179475,2.199625,1.683764,0.107682,1.718346,0.735869,...,5.961157,0.018577,1.017141,0.003584,-2.221883e+06,-51.965991,0.000283,-0.162587,-0.005935,0.017544
22320836,1.630780,85750.777531,-3.203427e-16,0.054975,8.229641,1.759578,1.511077,0.140114,1.586528,0.726964,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-0.001601,1.278782,0.008829,-0.021723
56131813,1.688159,100590.934263,-3.203427e-16,0.053591,37.481021,1.833218,1.631660,0.041272,1.642403,0.734819,...,-0.082819,0.011403,-0.241786,-0.003475,1.278301e+07,229.369008,-0.005528,1.112013,0.008574,-0.042068
37244770,1.741267,48615.898447,-3.203427e-16,0.046827,42.482157,1.907386,1.684634,0.040553,1.695071,0.822616,...,0.922495,-0.518547,1.870035,0.000564,-4.186273e+07,0.184928,-0.013964,0.359789,0.000599,0.021699
49344356,1.577195,46985.212288,-3.203427e-16,0.058672,5.493569,1.745115,1.449894,0.157792,1.543068,0.758444,...,4.413640,-0.007191,-0.055995,0.000013,5.950454e+05,9.670855,-0.000673,0.152857,0.001436,-0.009085
42120720,1.592838,43858.202699,-3.203427e-16,0.060431,6.841173,1.734866,1.465930,0.147654,1.545601,0.725131,...,-5.945015,-0.025132,0.873087,-0.007040,7.828450e+06,13.235655,-0.001086,-5.639784,-0.011643,-0.070389
51524302,1.650320,50872.684947,-3.203427e-16,0.037030,50.801961,1.754263,1.612324,0.026148,1.617883,0.943735,...,0.659891,-0.047359,0.520184,0.000124,3.477882e+06,11.190002,-0.013543,0.144951,0.000093,-0.062825
47451087,1.731607,58265.441994,-3.203427e-16,0.034513,45.091798,1.937058,1.692823,0.027326,1.699336,0.909987,...,-2.242555,0.214013,0.691563,0.000913,6.537215e+05,15.180199,0.000160,-0.369953,-0.001383,-0.016435
53004114,1.765041,47112.717625,6.428923e-03,0.061759,36.029139,3.408781,1.646975,0.057774,1.647673,0.728600,...,3.255458,-0.092059,1.468045,-0.000285,1.216261e+06,29.426239,-0.026447,3.760034,0.002288,-0.298484


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [17]:
X_train

,post_exponential_firstorder_90Percentile,post_exponential_firstorder_Energy,post_exponential_firstorder_Entropy,post_exponential_firstorder_InterquartileRange,post_exponential_firstorder_Kurtosis,post_exponential_firstorder_Maximum,post_exponential_firstorder_Mean,post_exponential_firstorder_MeanAbsoluteDeviation,post_exponential_firstorder_Median,post_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,-0.356290,-0.731212,-0.252203,0.293538,-1.114044,-0.008328,-0.562625,-0.073524,-0.791088,0.594033,...,1.041185,0.430590,1.417986,2.970664,-0.773458,0.121528,0.112509,0.284331,0.144845,0.095695
37567637,-1.252132,-0.568393,-0.227698,-0.609097,3.852592,0.483972,-1.022962,-0.893341,-1.315827,1.072583,...,-0.729295,1.800501,-0.849912,-1.609575,1.157863,0.044854,0.112509,-0.674535,-1.007998,0.341880
49087653,-0.775661,0.108748,-0.252203,-0.399425,1.625417,-0.714234,-0.636827,-0.607724,-0.827411,-0.763330,...,-0.548443,-0.054723,-1.072427,-0.161731,2.508987,0.040952,0.112509,-0.768167,-0.516646,0.570617
46695156,0.982961,-0.632229,-0.219394,0.785856,-1.106940,0.283744,0.412878,1.490557,0.969062,-0.591864,...,-3.702733,-0.043819,-1.425367,-2.357935,-0.218097,0.084438,0.112509,-2.711037,1.167611,-6.809536
54502130,0.580590,0.210915,0.689838,-0.237402,0.098615,1.789959,0.554888,-0.169646,0.380824,-0.695654,...,1.242198,-0.009661,-0.981194,-0.197446,-0.017367,0.108657,0.112509,1.451543,0.371115,-0.513785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,5.566203,0.338103,7.466637,6.077577,-1.270129,3.900957,2.478982,4.798796,0.903346,3.075564,...,-1.047106,-0.859898,-0.363735,-0.216729,-0.388629,0.146713,0.112509,-0.453397,0.951292,0.187747
37509338,-0.828746,-1.159343,-0.252203,-0.143504,-0.615337,-0.669462,-0.933286,-0.063273,-0.972647,-0.944829,...,0.201649,0.153311,2.963144,0.679053,-0.203761,0.065268,-8.888194,0.352873,0.159518,0.555886
50740947,0.406187,1.122067,-0.247199,-0.412377,0.730895,0.574460,0.769127,-0.396627,0.834476,-0.436753,...,-0.690885,-0.338318,1.614610,0.423637,1.177674,0.067552,0.112509,0.418199,0.136157,-0.396522


In [18]:
n_alphas = 100
alphas = np.logspace(-4, -1.2, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[0.0001     0.00010673 0.00011391 0.00012158 0.00012976 0.00013849
 0.00014781 0.00015775 0.00016837 0.0001797  0.00019179 0.0002047
 0.00021847 0.00023317 0.00024886 0.00026561 0.00028348 0.00030256
 0.00032292 0.00034465 0.00036784 0.00039259 0.00041901 0.0004472
 0.0004773  0.00050941 0.00054369 0.00058028 0.00061933 0.000661
 0.00070548 0.00075295 0.00080362 0.0008577  0.00091541 0.00097701
 0.00104275 0.00111292 0.00118781 0.00126774 0.00135305 0.0014441
 0.00154127 0.00164498 0.00175568 0.00187382 0.00199991 0.00213448
 0.00227812 0.00243141 0.00259502 0.00276965 0.00295602 0.00315493
 0.00336723 0.00359381 0.00383564 0.00409375 0.00436922 0.00466323
 0.00497702 0.00531193 0.00566938 0.00605088 0.00645804 0.00689261
 0.00735642 0.00785144 0.00837977 0.00894366 0.00954548 0.01018781
 0.01087336 0.01160504 0.01238595 0.01321941 0.01410896 0.01505836
 0.01607166 0.01715313 0.01830738 0.0195393  0.02085412 0.02225742
 0.02375514 0.02535364 0.02705972 0.02888059 0.03082399 0.03289817


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished


In [19]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 77.72
R squared test set -1.87


In [20]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.06309573444801933


In [21]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

49 ['post_exponential_glszm_HighGrayLevelZoneEmphasis'
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance'
 'post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis'
 'post_logarithm_gldm_LowGrayLevelEmphasis'
 'post_original_glszm_LowGrayLevelZoneEmphasis'
 'post_original_shape_Elongation' 'post_original_shape_Flatness'
 'post_square_glszm_ZoneVariance' 'post_wavelet-HLH_glcm_Autocorrelation'
 'post_wavelet-HLH_glcm_SumSquares' 'post_wavelet-HLL_firstorder_Minimum'
 'post_wavelet-HLL_firstorder_Range'
 'post_wavelet-HLL_glcm_MaximumProbability'
 'post_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized'
 'post_wavelet-LLH_glszm_GrayLevelVariance'
 'post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized'
 'delta_exponential_glrlm_LongRunHighGrayLevelEmphasis'
 'delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance'
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis'
 'delta_log-sigma-1-0-mm-3D_glszm_ZonePe

In [22]:
lasso_data = feature[feature_list]
lasso_data

,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_logarithm_gldm_LowGrayLevelEmphasis,post_original_glszm_LowGrayLevelZoneEmphasis,post_original_shape_Elongation,post_original_shape_Flatness,post_square_glszm_ZoneVariance,post_wavelet-HLH_glcm_Autocorrelation,...,delta_wavelet-HLH_firstorder_Median,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.150000,0.510204,0.244898,1.158957,0.252938,0.965909,0.721738,0.600507,0.000000e+00,2.227702,...,-0.000707,0.001723,-0.020000,-197.647973,-197.647973,0.026717,-0.001730,0.375000,0.217912,-0.063673
17815426,2.750000,0.509888,0.245056,0.103617,0.333720,0.683007,0.599704,0.420368,4.338772e+06,2.288415,...,-0.000882,-0.000932,-0.055521,3219.113800,3219.113800,-0.026725,-0.009022,0.701613,-0.252679,-0.124241
20109646,3.853659,0.546855,0.226572,0.433379,0.280026,0.338235,0.594562,0.536612,1.325722e+08,2.242186,...,-0.000066,-0.000249,-0.163265,-2598.354714,-2598.354714,0.006331,-0.005611,0.133333,-0.059095,0.044766
22320836,1.000000,0.539776,0.230112,0.727291,0.190972,0.625000,0.686474,0.441657,0.000000e+00,2.245474,...,-0.000094,-0.004728,-0.125000,-1988.693499,-1988.693499,-0.026800,-0.005635,-0.166667,-0.040512,0.038058
32221501,1.000000,0.654321,0.172840,0.451359,0.241522,0.750000,0.924649,0.261922,0.000000e+00,2.246201,...,0.000684,0.005952,0.097222,-695.601258,-695.601258,-0.008834,-0.015482,0.750000,-0.178836,0.057469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.000000,0.502959,0.248521,0.897160,0.251061,1.000000,0.661731,0.565288,0.000000e+00,2.253798,...,0.003185,-0.000008,0.000000,-169.814090,-169.814090,-0.014890,-0.010633,0.750000,0.092998,0.065063
56124435,1.000000,0.507200,0.246400,0.290226,0.116473,0.916667,0.520381,0.209350,0.000000e+00,2.246883,...,-0.000122,-0.001687,-0.009796,-433.785447,-433.785447,0.003013,0.001943,-0.750000,0.087433,0.005844
56131318,2.500000,0.500094,0.249953,0.879929,0.211502,0.817901,0.612600,0.254582,0.000000e+00,2.234573,...,0.000025,0.001527,-0.052597,3111.624309,3111.624309,0.000017,0.002660,0.666667,0.144587,0.003868


In [23]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_logarithm_gldm_LowGrayLevelEmphasis,post_original_glszm_LowGrayLevelZoneEmphasis,post_original_shape_Elongation,post_original_shape_Flatness,post_square_glszm_ZoneVariance,post_wavelet-HLH_glcm_Autocorrelation,...,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.150000,0.510204,0.244898,1.158957,0.252938,0.965909,0.721738,0.600507,0.000000e+00,2.227702,...,0.001723,-0.020000,-197.647973,-197.647973,0.026717,-0.001730,0.375000,0.217912,-0.063673,1.0
17815426,2.750000,0.509888,0.245056,0.103617,0.333720,0.683007,0.599704,0.420368,4.338772e+06,2.288415,...,-0.000932,-0.055521,3219.113800,3219.113800,-0.026725,-0.009022,0.701613,-0.252679,-0.124241,2.0
20109646,3.853659,0.546855,0.226572,0.433379,0.280026,0.338235,0.594562,0.536612,1.325722e+08,2.242186,...,-0.000249,-0.163265,-2598.354714,-2598.354714,0.006331,-0.005611,0.133333,-0.059095,0.044766,0.0
22320836,1.000000,0.539776,0.230112,0.727291,0.190972,0.625000,0.686474,0.441657,0.000000e+00,2.245474,...,-0.004728,-0.125000,-1988.693499,-1988.693499,-0.026800,-0.005635,-0.166667,-0.040512,0.038058,1.0
32221501,1.000000,0.654321,0.172840,0.451359,0.241522,0.750000,0.924649,0.261922,0.000000e+00,2.246201,...,0.005952,0.097222,-695.601258,-695.601258,-0.008834,-0.015482,0.750000,-0.178836,0.057469,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,1.000000,0.502959,0.248521,0.897160,0.251061,1.000000,0.661731,0.565288,0.000000e+00,2.253798,...,-0.000008,0.000000,-169.814090,-169.814090,-0.014890,-0.010633,0.750000,0.092998,0.065063,2.0
56124435,1.000000,0.507200,0.246400,0.290226,0.116473,0.916667,0.520381,0.209350,0.000000e+00,2.246883,...,-0.001687,-0.009796,-433.785447,-433.785447,0.003013,0.001943,-0.750000,0.087433,0.005844,1.0
56131318,2.500000,0.500094,0.249953,0.879929,0.211502,0.817901,0.612600,0.254582,0.000000e+00,2.234573,...,0.001527,-0.052597,3111.624309,3111.624309,0.000017,0.002660,0.666667,0.144587,0.003868,2.0


In [24]:
clinical_train = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv", index_col=0) 
clinical_test = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0) 
clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [25]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['post_exponential_glszm_HighGrayLevelZoneEmphasis',
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance',
 'post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis',
 'post_logarithm_gldm_LowGrayLevelEmphasis',
 'post_original_glszm_LowGrayLevelZoneEmphasis',
 'post_original_shape_Elongation',
 'post_original_shape_Flatness',
 'post_square_glszm_ZoneVariance',
 'post_wavelet-HLH_glcm_Autocorrelation',
 'post_wavelet-HLH_glcm_SumSquares',
 'post_wavelet-HLL_firstorder_Minimum',
 'post_wavelet-HLL_firstorder_Range',
 'post_wavelet-HLL_glcm_MaximumProbability',
 'post_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized',
 'post_wavelet-LLH_glszm_GrayLevelVariance',
 'post_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized',
 'delta_exponential_glrlm_LongRunHighGrayLevelEmphasis',
 'delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance',
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis',
 'delta_log-sigma-1-

In [26]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_logarithm_gldm_LowGrayLevelEmphasis,post_original_glszm_LowGrayLevelZoneEmphasis,post_original_shape_Elongation,post_original_shape_Flatness,post_square_glszm_ZoneVariance,...,delta_wavelet-HLH_firstorder_Median,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,3.0,0.563776,0.218112,0.683121,0.340917,0.765873,0.824313,0.724408,4.088324e+07,...,-0.000008,-0.000504,0.000000,509.711550,509.711550,-0.007470,0.005971,1.238095,0.012771,0.001135
38924846,0.0,1.0,0.882812,0.058594,1.197356,0.251090,0.750000,0.461271,0.223685,0.000000e+00,...,-0.001229,0.004004,0.146924,-288.284702,-288.284702,-0.059519,-0.006989,-0.900000,-0.074180,0.010441
56241796,1.0,1.0,0.722222,0.138889,1.197028,0.154028,0.962500,0.508331,0.241473,0.000000e+00,...,-0.000219,0.000158,0.003685,418.441321,418.441321,0.014821,0.004977,0.200000,-0.226132,-0.130211
40815882,1.0,1.0,0.722222,0.138889,0.500025,0.249874,0.625000,0.861895,0.447454,0.000000e+00,...,0.000061,-0.002678,0.055556,31.474672,31.474672,-0.043059,-0.003146,1.050000,0.190010,-0.052852
33908849,0.0,3.0,0.543253,0.228374,0.569428,0.174866,0.578704,0.748302,0.635857,1.572119e+08,...,-0.000127,-0.000435,0.000000,-777.711610,-777.711610,0.017315,-0.012366,-0.214286,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.0,0.516868,0.241566,0.999313,0.149120,0.750000,0.818625,0.478301,0.000000e+00,...,0.000016,-0.002595,0.077639,23.327664,23.327664,0.005043,-0.007047,0.088235,0.088587,0.088412
49087653,1.0,1.0,0.598765,0.200617,1.481504,0.255060,0.812500,0.632719,0.211145,0.000000e+00,...,-0.000263,0.001285,0.071837,508.588955,508.588955,-0.002807,-0.000420,0.589286,-0.135015,-0.022400
49588132,0.0,1.0,0.510204,0.244898,1.311763,0.068456,0.625000,0.512104,0.210427,0.000000e+00,...,-0.000740,0.000594,0.000000,431.863021,431.863021,-0.003426,0.000383,0.200000,-0.076182,0.092368


In [27]:
train_data = clinical_train
test_data = clinical_test

In [28]:
train_data

,encoded_PDL1,post_exponential_glszm_HighGrayLevelZoneEmphasis,post_log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized,post_log-sigma-1-0-mm-3D_glszm_GrayLevelVariance,post_log-sigma-1-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,post_logarithm_gldm_LowGrayLevelEmphasis,post_original_glszm_LowGrayLevelZoneEmphasis,post_original_shape_Elongation,post_original_shape_Flatness,post_square_glszm_ZoneVariance,...,delta_wavelet-HLH_firstorder_Median,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,3.0,0.563776,0.218112,0.683121,0.340917,0.765873,0.824313,0.724408,4.088324e+07,...,-0.000008,-0.000504,0.000000,509.711550,509.711550,-0.007470,0.005971,1.238095,0.012771,0.001135
38924846,0.0,1.0,0.882812,0.058594,1.197356,0.251090,0.750000,0.461271,0.223685,0.000000e+00,...,-0.001229,0.004004,0.146924,-288.284702,-288.284702,-0.059519,-0.006989,-0.900000,-0.074180,0.010441
56241796,1.0,1.0,0.722222,0.138889,1.197028,0.154028,0.962500,0.508331,0.241473,0.000000e+00,...,-0.000219,0.000158,0.003685,418.441321,418.441321,0.014821,0.004977,0.200000,-0.226132,-0.130211
40815882,1.0,1.0,0.722222,0.138889,0.500025,0.249874,0.625000,0.861895,0.447454,0.000000e+00,...,0.000061,-0.002678,0.055556,31.474672,31.474672,-0.043059,-0.003146,1.050000,0.190010,-0.052852
33908849,0.0,3.0,0.543253,0.228374,0.569428,0.174866,0.578704,0.748302,0.635857,1.572119e+08,...,-0.000127,-0.000435,0.000000,-777.711610,-777.711610,0.017315,-0.012366,-0.214286,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.0,0.516868,0.241566,0.999313,0.149120,0.750000,0.818625,0.478301,0.000000e+00,...,0.000016,-0.002595,0.077639,23.327664,23.327664,0.005043,-0.007047,0.088235,0.088587,0.088412
49087653,1.0,1.0,0.598765,0.200617,1.481504,0.255060,0.812500,0.632719,0.211145,0.000000e+00,...,-0.000263,0.001285,0.071837,508.588955,508.588955,-0.002807,-0.000420,0.589286,-0.135015,-0.022400
49588132,0.0,1.0,0.510204,0.244898,1.311763,0.068456,0.625000,0.512104,0.210427,0.000000e+00,...,-0.000740,0.000594,0.000000,431.863021,431.863021,-0.003426,0.000383,0.200000,-0.076182,0.092368


In [29]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 50)
Unseen Data For Predictions: (22, 50)


In [30]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 50)"
4,Transformed data shape,"(101, 50)"
5,Transformed train set shape,"(79, 50)"
6,Transformed test set shape,"(22, 50)"
7,Numeric features,49
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

In [31]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [32]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

In [33]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.5200,0.7592,0.5200,0.5314,0.4432,0.1710,0.2366,0.1100
lr,Logistic Regression,0.5817,0.7301,0.5817,0.5935,0.5813,0.3503,0.3546,0.0480
rf,Random Forest Classifier,0.4683,0.6949,0.4683,0.4902,0.4273,0.1252,0.1576,0.2340
nb,Naive Bayes,0.5567,0.6940,0.5567,0.5913,0.5483,0.3430,0.3602,0.0380
et,Extra Trees Classifier,0.5567,0.6920,0.5567,0.5614,0.5458,0.2925,0.3022,0.2680
knn,K Neighbors Classifier,0.5183,0.6814,0.5183,0.5368,0.5023,0.2538,0.2613,0.0640
lightgbm,Light Gradient Boosting Machine,0.5433,0.6791,0.5433,0.5609,0.5305,0.2755,0.2867,0.4960
lda,Linear Discriminant Analysis,0.5175,0.6514,0.5175,0.5167,0.5042,0.2737,0.2813,0.0500
gbc,Gradient Boosting Classifier,0.4925,0.6248,0.4925,0.5236,0.4815,0.1737,0.1822,0.6220
qda,Quadratic Discriminant Analysis,0.3925,0.5683,0.3925,0.4631,0.3556,0.1378,0.1586,0.0360


[AdaBoostClassifier(algorithm='SAMME.R', base_estimator='deprecated',
                    estimator=None, learning_rate=1.0, n_estimators=50,
                    random_state=3457),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='sqrt',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        n_estimators=100, n_jobs=-1, oob_score=False,
                        random_state=3457, verbose=0,

In [34]:
lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.1875,0.3828,0.1875,0.1750,0.1806,-0.3000,-0.3019
1,0.6875,0.8067,0.6875,0.7359,0.6927,0.5152,0.5249
2,0.6250,0.7446,0.6250,0.6399,0.5854,0.4545,0.4878
3,0.7500,0.8438,0.7500,0.7750,0.7569,0.6168,0.6205
4,0.4667,0.6174,0.4667,0.4788,0.4370,0.0909,0.1021
Mean,0.5433,0.6791,0.5433,0.5609,0.5305,0.2755,0.2867
Std,0.2013,0.1669,0.2013,0.2183,0.2058,0.3380,0.3433


In [35]:
prediction = predict_model(lightgbm, data=test_data)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.5909,0.5900,0.5909,0.6061,0.5893,0.3487,0.3536
